# Mongo aggregate

## 1. Setup

### 1.1. Utils

In [ ]:
from bson import json_util as json
from pymongo.cursor import Cursor
from bson import datetime
import pytz

def format_result(rs):
    return '\t{}'.format(json.dumps(rs, indent=2).replace('\n', '\n\t'))

def date_to_datetime(date):
    return datetime.datetime.combine(date, datetime.datetime.min.time()).replace(tzinfo=pytz.utc)

### 1.2. Make connection

In [ ]:
from pymongo import MongoClient, ASCENDING, DESCENDING, IndexModel

mongo = MongoClient(host='localhost', port=27017, maxPoolSize=5)
print('* mongodb connect success: {}'.format(mongo))

### 1.3. Get or create db and collection

In [ ]:
user_collection = user_collection = mongo['test_db']['user']
print('* user collection is: {}'.format(user_collection))

## 2. Pipeline stage

### 2.1. `$project` stage

- Define which attributes should include in result

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'homework': [10, 5, 10], 'quiz': [10, 8], 'extraCredit': 0},
    {'id': 2, 'name': 'Emma', 'homework': [5, 5, 7], 'quiz': [6, 7], 'extraCredit': 3},
    {'id': 3, 'name': 'Lucy', 'homework': [6, 7, 5], 'quiz': [9, 8], 'extraCredit': 4}
])


stage = {
    '$project': {
        'id': 1,
        'name': 1,
        'extraCredit': 1
    }
}
with user_collection.aggregate([stage]) as rs:
    print('* project include "id", "name" and "extraCredit":\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.2. `$addFields` stage

- add some fileds in result

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'homework': [10, 5, 10], 'quiz': [10, 8], 'extraCredit': 0},
    {'id': 2, 'name': 'Emma', 'homework': [5, 5, 7], 'quiz': [6, 7], 'extraCredit': 3},
    {'id': 3, 'name': 'Lucy', 'homework': [6, 7, 5], 'quiz': [9, 8], 'extraCredit': 4}
])

stage_1 = {
    '$addFields': {
        'totalHomework': {'$sum': "$homework"},
        'totalQuiz': {'$sum': "$quiz"}
    }
}

stage_2 = {
    '$addFields': {
        'totalScore': {
            '$add': ['$totalHomework', '$totalQuiz', '$extraCredit']
        }
    }
}

stage_3 = {
    '$project': {
        'id': 1,
        'name': 1,
        'totalScore': 1
    }
}

with user_collection.aggregate([stage_1, stage_2, stage_3]) as rs:
    print('* show result after add files:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.3. `$match` stage

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 26}
])

stage_1 = {
    '$group': {
        '_id': '$gender',
        'names': {'$addToSet': '$name'},
        'totalAge': {'$sum': "$age"}
    }
}

stage_2 = {
    '$match': {
        'totalAge': {
            '$gt': 50
        }
    }
}

with user_collection.aggregate([stage_1, stage_2]) as rs:
    print('* show result after add to set:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.4. `$bucket` and `$bucketAuto` stage

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 26}
])

stage = {
    '$bucket': {
        'groupBy': '$gender',
        'boundaries': ['F', 'M', 'U'],
        'default': 'U',
        'output': {
            "count": {'$sum': 1},
            "users": {
                '$push': {
                    'name': '$name',
                    'gender': '$gender',
                    'age': '$age'
                }
            }
        }
    }
}

with user_collection.aggregate([stage]) as rs:
    print('* show result after bucket:\n{}'.format(format_result(rs)))


stage = {
    '$bucketAuto': {
        'groupBy': '$gender',
        'buckets': 2,
        'output': {
            "count": {'$sum': 1},
            "users": {
                '$push': {
                    'name': '$name',
                    'gender': '$gender',
                    'age': '$age'
                }
            }
        }
    }
}

with user_collection.aggregate([stage]) as rs:
    print('\n* show result after auto bucket:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.5. `$collStats` stage

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 26}
])

stage = {
    'latencyStats': {'histograms': True},
    'storageStats': {'scale': 11}
}

with user_collection.aggregate([stage]) as rs:
    print('* show result after bucket:\n{}'.format(format_result(rs)))

## 3. Pipline operators

### 3.1. `$add` and `$sum` operator

- Add certain fields

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'scores': [10, 5, 10], 'quiz': [10, 8], 'extraCredit': 0},
    {'id': 2, 'name': 'Emma', 'scores': [5, 5, 7], 'quiz': [6, 7], 'extraCredit': 3},
    {'id': 3, 'name': 'Lucy', 'scores': [6, 7, 5], 'quiz': [9, 8], 'extraCredit': 4}
])


pipline_1 = {
    '$project': {
        'id': 1,
        'name': 1,
        'totalScore': {'$sum': ['$scores']},
        'totalQuiz': {'$sum': ['$quiz']},
        'extraCredit': 1
    }
}

pipline_2 = {
    '$project': {
        'id': 1,
        'name': 1,
        'finalScore': {'$add': ['$totalScore', '$totalQuiz', '$extraCredit']}
    }
}
with user_collection.aggregate([pipline_1, pipline_2]) as rs:
    print('* project include "id", "name" and "extraCredit":\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 3.2. `$addToSet` operator

- add more items into a field as set

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 26}
])

pipline = {
    '$group': {
        '_id': '$gender',
        'ages': {'$addToSet': "$age"}
    }
}
with user_collection.aggregate([pipline]) as rs:
    print('* show result after add to set:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 3.3. `$and`, `$eq`, `$gt`, `$lt`, `$gte` and `$lte` Operator

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 26}
])

pipline = {
    '$project': {
        'id': 1,
        'name': 1,
        'gender': 1,
        'age': 1,
        'qualified': {'$and': [{'$eq': ['$gender', 'F']}, {'$gt': ['$age', 30]}]}
    }
}
with user_collection.aggregate([pipline]) as rs:
    print('* show result after add to set:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

### 2.1. Count by condition

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])


condition = {'id': {'$gt': 1}}
count = user_collection.count_documents(condition)
print('* count by condition "{}" is: {}'.format(json.dumps(condition), count))


user_collection.delete_many({})

### 2.2. Count by group

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M'},
    {'id': 2, 'name': 'Emma', 'gender': 'F'},
    {'id': 3, 'name': 'Lucy', 'gender': 'F'}
])


condition = {
    '$group': {
        '_id': '$gender', 
        'count': {'$sum': 1}
    }
}
with user_collection.aggregate([condition]) as rs:
    print('* each count of group by gender is:\n{}'.format(format_result(rs)))


user_collection.delete_many({})

## 3. Sum

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 21}
])


condition = {
    '$group': {
        '_id': '$gender',
        'count': {'$sum': 1},
        'total_age': {'$sum': '$age'}
    }
}

with user_collection.aggregate([condition]) as rs:
    print('* aggregate result is: {}'.format(format_result(rs)))


user_collection.delete_many({})

## 4. Max and min

In [ ]:
user_collection.insert_many([
    {'id': 1, 'name': 'Alvin', 'gender': 'M', 'age': 38},
    {'id': 2, 'name': 'Emma', 'gender': 'F', 'age': 35},
    {'id': 3, 'name': 'Lucy', 'gender': 'F', 'age': 21}
])

condition = {
    '$group': {
        '_id': '$gender',
        'max_age': {'$max': '$age'},
        'min_age': {'$min': '$age'}
    }
}

with user_collection.aggregate([condition]) as rs:
    print('* aggregate result is: {}'.format(format_result(rs)))


user_collection.delete_many({})